### Edgar Moises Hernandez-Gonzalez
#### 19/11/19-07/02/20
#### Clasificacion de RWE-DWT de MI-EEG con CNN
##### Validacion cruzada de 5
##### Optimizar hiperparametros con busqueda de cuadricula

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam

In [0]:
x_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RWE_DWT/MI-EEG-RWE-A01T.csv", header=None)
x_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RWE_DWT/MI-EEG-RWE-A01E.csv", header=None)
y_train = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RWE_DWT/EtiquetasA01T.csv", header=None)
y_test = pd.read_csv("/content/drive/My Drive/BCI-IV-2a/RWE_DWT/EtiquetasA01E.csv", header=None)

In [6]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(288, 66)
(288, 66)
(288, 1)
(288, 1)


In [7]:
x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))

print(x.shape)
print(y.shape)

(576, 66)
(576, 1)


In [8]:
n_samples = len(y)

print("n_samples:", n_samples)

n_samples: 576


In [9]:
n_classes = len(np.unique(y_test)) #esto se podria calcular asi n_clases = len(np.unique(y)), la y puede ser train o test

print("n_classes:", n_classes)

n_classes: 4


In [0]:
#escalar los datos para que tengan media = 0 y desviacion estandar = 1
x = StandardScaler().fit_transform(x)

In [11]:
#codificar etiquetas con valores de 0 a nClases-1
y = LabelEncoder().fit_transform(y)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
#convertir de 2D a 4D
x = x.reshape((n_samples, 22, 3, 1))

print(x.shape)

(576, 22, 3, 1)


In [0]:
def CNN_2D(n_filtros, tamano_filtros, n_neuronas, p_dropout, tasa_aprendizaje):
    model = Sequential()
    model.add(Conv2D(n_filtros, tamano_filtros, activation='relu', input_shape=(22,3,1)))
    model.add(Flatten())
    model.add(Dense(n_neuronas, activation='relu'))
    model.add(Dropout(p_dropout))
    model.add(Dense(n_classes, activation='softmax'))
    optimizer = Adam(lr=tasa_aprendizaje)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [0]:
clf = scikit_learn.KerasClassifier(build_fn = CNN_2D, epochs=100, batch_size=18, verbose=0)

In [0]:
n_filtros = [4, 64]
tamano_filtros = [(3,3), [11,3]]
n_neuronas = [32, 64, 128]
p_dropout = [0.2, 0.5]
tasa_aprendizaje = [1e-4, 1e-5, 1e-6]
hiperparametros = dict(n_filtros=n_filtros, tamano_filtros=tamano_filtros,
                       n_neuronas=n_neuronas, p_dropout=p_dropout,
                       tasa_aprendizaje=tasa_aprendizaje)

In [17]:
2*2*3*2*3

72

In [0]:
clf = GridSearchCV(clf, hiperparametros, n_jobs=-1, cv=5)

In [19]:
clf.fit(x, y)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


RuntimeError: ignored

In [20]:
print(clf.best_params_)
print(clf.best_score_)

{'n_filtros': 64, 'n_neuronas': 128, 'p_dropout': 0.5, 'tamano_filtros': [11, 3], 'tasa_aprendizaje': 0.0001}
0.5989805221557617


In [21]:
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))

AttributeError: ignored